In [162]:
# with open("temp.txt","w+") as file:
#     for _,sub in subsidies.iterrows():
#         file.writelines(f"Scheme Name:\n{sub['Subsidy scheme']} \n")
#         file.writelines(f"Scheme Discription:\n{sub['Description']} \n\n")

In [163]:
import pandas as pd
# from langchain import PromptTemplate
import json

user_details={
'gender':'female',
'scheme_type':['Women'],
'state':'Karnataka',
'occupation':'artisan',
}

subsidies=pd.read_csv("./processed_subsidies.csv")
subsidies=subsidies[(subsidies["Scheme Type"].isin(user_details['scheme_type'])) & (subsidies["Applicability Central/State"].isin(["Central",user_details["state"]]))]
subsidies_str=subsidies[["Subsidy scheme","eligibility_criteria"]].to_csv(index=False)

selected_user_details={key: user_details[key] for key in ["gender", "state", "occupation"]}

user_details_str = json.dumps(selected_user_details, indent=4)

llm_prompt_template = f"""You are an advisor who checks if the the user is eligible for any subsidy schemes listed below or not. \
Government subsidy programs: {subsidies_str} \
User details: {user_details_str}\
If there is no eligible subsidy, simply answer 'not elibible for any scheme'. \
Else return only the scheme names of all the eligible programs, without any explaination. \
List down the names of all the eligible schemes with * as marker for each scheme.\
Make sure to not assume any thing, and answer concisely.\
Answer concisely, with only the data provided. Do not add any additional information.\
"""

# llm_prompt = PromptTemplate.from_template(llm_prompt_template)


In [164]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="llama3") 

response=model.invoke(llm_prompt_template).strip("'")

response

'* TREAD\n* Exemption from Stamp Duty\n* Concessional Registration Charges\n* Reimbursement of Non-Agriculture Conversion Fee\n* Exemption from Entry Tax\n* Subsidy for setting up Effluent Treatment Plant (ETP)\n* Interest Subsidy for Micro Enterprises'

In [165]:
for i in response.split("\n"):
    print(i)

* TREAD
* Exemption from Stamp Duty
* Concessional Registration Charges
* Reimbursement of Non-Agriculture Conversion Fee
* Exemption from Entry Tax
* Subsidy for setting up Effluent Treatment Plant (ETP)
* Interest Subsidy for Micro Enterprises


In [166]:
eligible_schemes=response.split("*")[1:]
for i in range(len(eligible_schemes)):
    eligible_schemes[i]=eligible_schemes[i].strip(" \n\'\"")
eligible_schemes

['TREAD',
 'Exemption from Stamp Duty',
 'Concessional Registration Charges',
 'Reimbursement of Non-Agriculture Conversion Fee',
 'Exemption from Entry Tax',
 'Subsidy for setting up Effluent Treatment Plant (ETP)',
 'Interest Subsidy for Micro Enterprises']

In [45]:
from langchain_community.chat_models import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_experimental.text_splitter import SemanticChunker
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import pandas as pd
import faiss
import numpy as np

docs=pd.read_csv("processed_subsidies.csv").to_csv(index=False)

doc = Document(page_content=docs)
            
# Split document into chunks
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents([doc])
# chunks = SemanticChunker(OllamaEmbeddings(model="llama3")).split_documents([doc])

# Extract text content from Document objects
texts = [chunk.page_content for chunk in chunks]
np.save('texts.npy', texts)
            
# Create embeddings
embeddings = OllamaEmbeddings(model="llama3")
embedding_vectors = np.array([embeddings.embed_documents([text])[0] for text in texts], dtype=np.float32)
            
# Initialize FAISS index
dimension = embedding_vectors.shape[1]  # Dimensionality of the embedding vectors
index = faiss.IndexFlatL2(dimension)  # Using L2 distance for similarity
index.add(embedding_vectors)  # Add vectors to the index
            

# Save the index to disk
faiss.write_index(index, 'vector_db.index')
print("Vector database created and saved to 'vector_db.index'")

Vector database created and saved to 'vector_db.index'


In [19]:
# import faiss
# import numpy as np

# # Load the index from disk
# index = faiss.read_index('vector_db.index')
# print("Vector database loaded from 'vector_db.index'")


Vector database loaded from 'vector_db.index'


In [21]:
# query="Is there any subsidy scheme for a low income farmer ?"

# query_vector = embeddings.embed_documents([query])[0]
# query_vector = np.array([query_vector], dtype=np.float32)

# k = 5  # number of nearest neighbors
# D, I = index.search(query_vector, k)  # D: distances, I: indices

# print(f"Nearest neighbors indices: {I}")
# print(f"Distances: {D}")

# retrieved_texts = [texts[idx] for idx in I[0]]
# retrieved_texts

Nearest neighbors indices: [[358  14 193 128 117]]
Distances: [[22985.398 23461.973 24395.008 24603.648 26015.02 ]]


['Private Developer will allot the shed to SC/ST entrepreneur in the mini estate developed by him and approved under state government scheme.',
 'private industrial estates /parks/food parks,SPV formed by GoK/GoI,and other approved industrial parks shall be exempted as below:',
 'The component proposes to provide financial assistance to the Apex Co-operative Societies, Central Co-op. Societies, Primary Co-operative Societies, Public Sector Enterprises, franchisees appointed by Coir Board in the coir industry and the Showroom and Sales Depots / Hindustan Coir of the Coir Board. The MDA is granted at the rate of 10% of their average annual sales turnover of coir products including coir fibre, coir pith, coir pith block, coir pith products, coir pith organic manure, coir yarn, mats, matting, rubberized coir goods, coir geotextile, garden articles, coir bags, coir umbrella, coir chapel, coir ornaments, coir handicrafts, coir wood and other innovative products during the preceding three fin

In [44]:
from langchain_ollama.llms import OllamaLLM
import pandas as pd
import faiss
import numpy as np
from langchain_ollama import OllamaEmbeddings


def chat_bot_subsidies(query):
    model = OllamaLLM(model="llama3") 
    embeddings = OllamaEmbeddings(model="llama3")
    index = faiss.read_index('vector_db.index')
    texts = np.load('texts.npy', allow_pickle=True)
    query_vector = embeddings.embed_documents([query])[0]
    query_vector = np.array([query_vector], dtype=np.float32)

    k = 5  # number of nearest neighbors
    D, I = index.search(query_vector, k)  # D: distances, I: indices
    retrieved_texts = [texts[idx] for idx in I[0]]

    llm_prompt_template = f"""You are an assistant who needs to answer to the user queries. \
    Retrieved Context: {retrieved_texts} \
    User questions: {query}\
    Make sure to respond only from the information provided \
    Do not assume any data. \
    If the question is unrelated to the context, simply return "Sorry, I am not able to respond to your query !" only. Do not add any additional details and explaination. \
    Make sure to not add any additional data and notes.
    """

    response=model.invoke(llm_prompt_template)

    return response

In [49]:
print(chat_bot_subsidies("tell me about subsidies for msme by center"))

According to the provided context, subsidies for MSMEs by the Center include:

* 6% interest subsidy on term loan amount up to INR 25 lakh p.a. for 5 years
* Additional incentive of 1% interest subsidy to registered start-ups in service sector and having Udyam and MSME registration

Please note that these subsidies are subject to specific eligibility criteria mentioned in the context.
